# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from statsmodels.miscmodels.ordinal_model import OrderedModel

# Setting up model dataframe

In [2]:
df2020 = pd.read_csv("data/df2020.csv")
df2021 = pd.read_csv("data/df2021.csv")
df2022 = pd.read_csv("data/df2022.csv")

In [3]:
df2020.drop(columns = df2020.columns[0], axis = 1, inplace = True)
df2021.drop(columns = df2021.columns[0], axis = 1, inplace = True)
df2022.drop(columns = df2022.columns[0], axis = 1, inplace = True)

In [4]:
df2020['Year'] = 2020
df2021['Year'] = 2021
df2022['Year'] = 2022

In [5]:
df = pd.concat([df2020,df2021,df2022])

In [6]:
df.to_csv('data/combineddata.csv', index = False)

In [7]:
df.columns

Index(['Player', 'Round', 'DraftNumber', 'PreTeam', 'urlsummary', 'PreType',
       'GP', 'GS', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%',
       'FTM', 'FTA', 'FT%', 'OFF', 'DEF', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'Position', 'Year'],
      dtype='object')

In [8]:
df = pd.read_csv('data/combineddata.csv')

In [19]:
modeldf = df.drop(columns = ['Player','Round','PreTeam','urlsummary'], axis = 1)

In [20]:
modeldf.reset_index(inplace = True)

In [21]:
modeldf = modeldf.drop(modeldf[modeldf['GP']==0].index)

In [22]:
modeldf.reset_index(inplace = True)

In [23]:
modeldf.drop(columns = 'index', axis = 1, inplace = True)

In [24]:
modeldf.drop(columns = 'Year', axis = 1, inplace = True)

In [27]:
modeldf.drop(columns = 'level_0', axis = 1, inplace = True)

In [29]:
modeldf.to_csv('data/modeldf.csv', index = False)

# Pre-processing

In [31]:
modeldf = pd.read_csv('data/modeldf.csv')

In [33]:
pos2 = []
for i in range(len(modeldf)):
    if modeldf['Position'][i] == 'G':
        pos2.append('PG')
    elif modeldf['Position'][i] == 'F':
        pos2.append('SF')
    elif modeldf['Position'][i] == 'FC':
        pos2.append('PF')
    elif modeldf['Position'][i] == 'GF':
        pos2.append('SG')
    else:
        pos2.append(modeldf['Position'][i])


In [34]:
pos3 = []
for j in range(len(modeldf)):
    if modeldf['Position'][j] == 'G':
        pos3.append('SG')
    elif modeldf['Position'][j] == 'F':
        pos3.append('PF')
    elif modeldf['Position'][j] == 'FC':
        pos3.append('C')
    elif modeldf['Position'][j] == 'GF':
        pos3.append('SF')
    else:
        pos3.append(modeldf['Position'][j])

In [35]:
modeldf['Position2'] = pos2
modeldf['Position3'] = pos3

In [36]:
pos4 = []
for k in range(len(modeldf)):
    if modeldf['Position2'][k] in ['PG', 'SG']:
        pos4.append('Backcourt')
    else:
        pos4.append('Frontcourt')

In [37]:
modeldf['Position4'] = pos4

In [38]:
categorical_columns = ['PreType', 'Position', 'Position2', 'Position3', 'Position4']

In [39]:
for col in categorical_columns:
    col_ohe = pd.get_dummies(modeldf[col], prefix = col)
    modeldf = pd.concat((modeldf,col_ohe), axis = 1).drop(col, axis = 1)

In [41]:
modeldf.to_csv('data/modeldfcategorical.csv',index = False)

# Training Ordinal Regression Model

In [2]:
modeldf = pd.read_csv('data/modeldfcategorical.csv')

In [3]:
#y = np.array(modeldf['DraftNumber'])
#X = np.array(modeldf.drop(columns = ['DraftNumber'], axis = 1))

y = modeldf['DraftNumber']
X = modeldf.drop(columns = ['DraftNumber'], axis = 1)

In [4]:
newy = pd.Series([1 if x<=30 else 2 for x in modeldf['DraftNumber']])

In [5]:
eff = (X['PTS']+ X['TRB'] + X['AST'] + X['STL'] + X['BLK'] - (X['FGA']-X['FGM']) - (X['FTA']-X['FTM']))/X['GP']

In [6]:
eff_df = pd.DataFrame({'EFF':eff})

In [11]:
newx = X.join(eff_df)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:,0:24],y, test_size = 0.25, random_state = 42)

In [9]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X.iloc[:,0:24],newy, test_size = 0.25, random_state = 42)

In [26]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(newx.iloc[:,0:21].join(newx.iloc[:,45:]),newy, test_size = 0.25, random_state = 42)

In [ ]:
# https://www.statsmodels.org/dev/examples/notebooks/generated/ordinal_regression.html#examples-notebooks-generated-ordinal-regression--page-root
# https://analyticsindiamag.com/a-complete-tutorial-on-ordinal-regression-in-python/

mod_prob = OrderedModel(y_train, X_train, distr = 'logit', hasconst= False)
res_prob = mod_prob.fit(method = 'bfgs')
pred = res_prob.model.predict(res_prob.params, exog = np.array(X_test))
pred_choice = pred.argmax(1)
print('Fraction of correct choice predictions')
print((np.asarray(y_test) == pred_choice).mean())

Optimization terminated successfully.
         Current function value: 3.922876
         Iterations: 257
         Function evaluations: 261
         Gradient evaluations: 261
Fraction of correct choice predictions
0.022727272727272728


In [14]:
mod_prob1 = OrderedModel(y_train1, X_train1, distr = 'logit', hasconst= False)
res_prob1 = mod_prob1.fit(method = 'bfgs')
pred1 = res_prob1.model.predict(res_prob1.params, exog = np.array(X_test1))
pred_choice1 = pred1.argmax(1)
print('Fraction of correct choice predictions')
print((np.asarray(y_test1) == pred_choice1).mean())

Optimization terminated successfully.
         Current function value: 0.628708
         Iterations: 166
         Function evaluations: 170
         Gradient evaluations: 170
Fraction of correct choice predictions
0.29545454545454547


In [27]:
mod_prob2 = OrderedModel(y_train2, X_train2, distr = 'logit', hasconst= False)
res_prob2 = mod_prob2.fit(method = 'bfgs')
pred2 = res_prob2.model.predict(res_prob2.params, exog = np.array(X_test2))
pred_choice2 = pred2.argmax(1)
print('Fraction of correct choice predictions')
print((np.asarray(y_test2) == pred_choice2).mean())

Optimization terminated successfully.
         Current function value: 0.621412
         Iterations: 162
         Function evaluations: 166
         Gradient evaluations: 166
Fraction of correct choice predictions
0.25


In [19]:
sigX = X[res_prob1.pvalues[res_prob1.pvalues < 0.5].index]

In [22]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(sigX,newy, test_size = 0.25, random_state = 42)

In [23]:
mod_prob3 = OrderedModel(y_train3, X_train3, distr = 'logit', hasconst= False)
res_prob3 = mod_prob3.fit(method = 'bfgs')
pred3 = res_prob3.model.predict(res_prob3.params, exog = np.array(X_test3))
pred_choice3 = pred3.argmax(1)
print('Fraction of correct choice predictions')
print((np.asarray(y_test3) == pred_choice3).mean())

Optimization terminated successfully.
         Current function value: 0.647643
         Iterations: 35
         Function evaluations: 38
         Gradient evaluations: 38
Fraction of correct choice predictions
0.20454545454545456


# Results

With y as draft picks, and x with 0 to 24: fraction of correct choice predictions is 0.02, add 43 and 44: model fails to find the maximum of loglikelihood function

With y as rounds, and x with 0 to 24: fraction of correct choice predictions is 0.29. Fails to converge if add columns 43 and 44. 

y as rounds, x 0 to 21, and EFF: fraction = 0.25. if 0 to 24 then fails to converge.

y as rounds, x are the significant features from the second model result summary. fraction correct 0.2

In [24]:
res_prob1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                      y   Log-Likelihood:                -82.989
Model:                   OrderedModel   AIC:                             216.0
Method:            Maximum Likelihood   BIC:                             288.0
Date:                Wed, 09 Nov 2022                                         
Time:                        17:01:07                                         
No. Observations:                 132                                         
Df Residuals:                     107                                         
Df Model:                          25                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
GP                  -0.0138      0.028     -0.485      0.628      -0.070       0.042
GS                   0.0352      0.029      1.205      0.228      -0.022       0.093
MIN                 -0.0261      0.054     -0.485      0.628      -0.131       0.079
PTS                  3.4127      4.779      0.714      0.475      -5.954      12.779
FGM                 -8.1153      9.498     -0.854      0.393     -26.730      10.499
FGA                  0.6253      0.533      1.174      0.240      -0.419       1.669
FG%                  0.5499      8.352      0.066      0.948     -15.820      16.920
3PM                 -2.0548      4.839     -0.425      0.671     -11.539       7.429
3PA                 -0.9212      0.568     -1.621      0.105      -2.035       0.193
3P%                  1.3413      2.005      0.669      0.504      -2.589       5.272
FTM                 -3.0751      5.079     -0.605      0.545     -13.030       6.880
FTA                 -0.0914      1.443     -0.063      0.949      -2.919       2.737
FT%                  0.7005      6.027      0.116      0.907     -11.112      12.513
OFF                 -9.6771     39.144     -0.247      0.805     -86.399      67.044
DEF                 -9.4557     39.127     -0.242      0.809     -86.143      67.232
TRB                  9.7288     39.118      0.249      0.804     -66.941      86.399
AST                  0.1708      0.201      0.851      0.395      -0.222       0.564
STL                 -0.8781      0.618     -1.421      0.155      -2.089       0.333
BLK                 -0.3610      0.478     -0.755      0.450      -1.298       0.576
TOV                 -0.1486      0.482     -0.308      0.758      -1.093       0.796
PF                  -0.2826      0.477     -0.593      0.553      -1.217       0.652
PreType_College      0.3849    215.358      0.002      0.999    -421.709     422.479
PreType_G League    -1.4391    215.374     -0.007      0.995    -423.565     420.687
PreType_Overseas     0.9225    215.357      0.004      0.997    -421.170     423.015
1/2                  0.1924    215.316      0.001      0.999    -421.820     422.204
====================================================================================
"""